In [0]:
# Constants
KAFKA_BROKER = "detrainingbg.adastragrp.com:9092"
KAFKA_TOPIC = "marketing.uk.netflix"
KAFKA_SSL_TRUSTSTORE_LOCATION = "abfss://source-data@sttrwesteu001.dfs.core.windows.net/cert/kafka.truststore.jks"
KAFKA_SSL_KEYSTORE_LOCATION = "abfss://source-data@sttrwesteu001.dfs.core.windows.net/cert/kafka.keystore.jks"
BRONZE_TABLE = "bronze.training_io.rt_marketing"
CHECKPOINT_PATH = "/Volumes/bronze/checkpoints/realtime"

# Function to retrieve secrets
def get_secrets(scope, username_key, password_key, key_store, private_key, trust_store):
    """
    Retrieve the Kafka credentials and SSL secrets from Databricks secrets scope.
    """
    username = dbutils.secrets.get(scope, username_key)
    password = dbutils.secrets.get(scope, password_key)
    key_store = dbutils.secrets.get(scope, key_store)
    private_key = dbutils.secrets.get(scope, private_key)
    trust_store = dbutils.secrets.get(scope, trust_store)
    return username, password, key_store, private_key, trust_store

# Retrieve Kafka credentials from secret scope
def retrieve_my_kafka_credentials():
    """
    Retrieve Kafka credentials and SSL certificates.
    """
    return get_secrets(
        "dbc-training-scope", 
        "KafkaUsername", 
        "KafkaPassword", 
        "KafkaSSLKeystorePassword", 
        "KafkaSSLPrivateKeyPassword", 
        "KafkaSSLTruststorePassword"
    )

# Function to configure Kafka options
def configure_kafka_stream(kafka_username, kafka_password, kafka_ssl_keystore_password, kafka_ssl_private_key_password, kafka_ssl_truststore_password):
    """
    Set up Kafka stream options for reading from a Kafka topic.
    """
    return (
        spark.readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", KAFKA_BROKER)
        .option("subscribe", KAFKA_TOPIC)
        .option("kafka.security.protocol", "SASL_SSL")
        .option("kafka.sasl.mechanism", "PLAIN")
        .option("kafka.ssl.endpoint.identification.algorithm", "")
        .option("kafka.sasl.jaas.config", f"""kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required 
                                                username={kafka_username} 
                                                password={kafka_password};""")
        .option("kafka.ssl.truststore.location", KAFKA_SSL_TRUSTSTORE_LOCATION)
        .option("kafka.ssl.truststore.password", kafka_ssl_truststore_password)
        .option("kafka.ssl.keystore.location", KAFKA_SSL_KEYSTORE_LOCATION)
        .option("kafka.ssl.keystore.password", kafka_ssl_keystore_password)
        .option("kafka.ssl.key.password", kafka_ssl_private_key_password)
        .option("startingOffsets", "earliest")
        .load()
    )

# Function to cast Kafka stream data
def cast_kafka_stream_data(kafka_df):
    """
    Cast the Kafka stream data to json format.
    """
    return kafka_df.selectExpr(
        "topic",
        "CAST(value AS STRING) as json_event",
        "partition",
        "offset",
        "timestamp"
    )

# Function to write the stream to a Delta table
def write_to_delta_stream(df, table_name, checkpoint_path):
    """
    Write the stream data to a Delta table.
    """
    df.writeStream \
        .format("delta") \
        .option("checkpointLocation", checkpoint_path) \
        .outputMode("append") \
        .trigger(processingTime="1 minute") \
        .toTable(table_name)

# Main execution flow
def main():
    # Retrieve Kafka credentials
    KafkaUsername, KafkaPassword, KafkaSSLKeystorePassword, KafkaSSLPrivateKeyPassword, KafkaSSLTruststorePassword = retrieve_my_kafka_credentials()
    
    # Configure Kafka stream
    kafka_stream_df = configure_kafka_stream(KafkaUsername, KafkaPassword, KafkaSSLKeystorePassword, KafkaSSLPrivateKeyPassword, KafkaSSLTruststorePassword)
    
    # Cast Kafka stream data
    df_casted = cast_kafka_stream_data(kafka_stream_df)
    
    # Write stream data to Delta table
    write_to_delta_stream(df_casted, BRONZE_TABLE, CHECKPOINT_PATH)

# Run the main function
if __name__ == "__main__":
    main()
